# Tutorial for binning data from the SXP instrument at the European XFEL

## Preparation
### Import necessary libraries

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import os
import xarray as xr
import numpy as np

from sed import SedProcessor
from sed.dataset import dataset

%matplotlib widget
import matplotlib.pyplot as plt

### Get data paths
The paths are such that if you are on Maxwell, it uses those. Otherwise data is downloaded in current directory from Zenodo.

Generally, if it is your beamtime, you can both read the raw data and write to processed directory. However, for the public data, you can not write to processed directory.

In [ ]:
beamtime_dir = "/gpfs/exfel/exp/SXP/202302/p004316/" # on Maxwell
if os.path.exists(beamtime_dir) and os.access(beamtime_dir, os.R_OK):
    path = beamtime_dir + "/raw/"
    buffer_path = "Au_Mica/processed/"
else:
    # data_path can be defined and used to store the data in a specific location
    dataset.get("Au_Mica") # Put in Path to a storage of at least 10 GByte free space.
    path = dataset.dir
    buffer_path = path + "/processed/"

### Config setup
Here we get the path to the config file and setup the relevant directories. This can also be done directly in the config file.

In [ ]:
# pick the default configuration file for SXP@XFEL
config_file = Path('../sed/config/sxp_example_config.yaml')
assert config_file.exists()

In [ ]:
# here we setup a dictionary that will be used to override the path configuration
config_override = {
    "core": {
        "paths": {
            "data_raw_dir": path,
            "data_parquet_dir": buffer_path,
        },
    },
}

### cleanup previous config files
In this notebook, we will show how calibration parameters can be generated. Therefore we want to clean the local directory of previously generated files.

**WARNING** running the cell below will delete the "sed_config.yaml" file in the local directory. If these contain precious calibration parameters, **DO NOT RUN THIS CELL**.

In [ ]:
local_folder_config = Path('./sed_config.yaml')
if local_folder_config.exists():
    os.remove(local_folder_config)
    print(f'deleted local config file {local_folder_config}')
assert not local_folder_config.exists()

## Load Au/Mica data
Now we load a couple of scans from Au 4f core levels. Data will be processed to parquet format first, if not existing yet, and then loaded into the processor.

In [ ]:
sp = SedProcessor(
    runs=["0058", "0059", "0060", "0061"],
    config=config_override,
    system_config=config_file,
    collect_metadata=False,
    verbose=True,
)

## Inspect the dataframe
We first try to get an overview of the structure of the data. For this, we look at the loaded dataframe:

In [ ]:
sp.dataframe.head()

### Train IDs in scans 
Next, let's look at the trainIDs contained in these runs

In [ ]:
plt.figure()
ids=sp.dataframe.trainId.compute().values
plt.plot(ids)
plt.show()

### Channel Histograms
Let's look at the single histograms of the main data channels

In [ ]:
sp.view_event_histogram(dfpid=3)

### PulseIds, ElectronIds
To get a better understanding of the structure of the data, lets look at the histograms of microbunches and electrons. We see that we have more hits at later microbunches, and only few multi-hits.

In [ ]:
axes = ["pulseId", "electronId"]
bins = [101, 11]
ranges = [(-0.5, 800.5), (-0.5, 10.5)]
sp.view_event_histogram(dfpid=1, axes=axes, bins=bins, ranges=ranges)

We can also inspect the counts per train as function of the trainId and the pulseId, which gives us a good idea about the evolution of the count rate over the run(s)

In [ ]:
plt.figure()
axes = ["trainId", "pulseId"]
bins = [100, 100]
ranges = [(ids.min()+1, ids.max()), (0, 800)]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)
res.plot()

## Spectrum vs. MicrobunchId
Let's check the TOF spectrum as function of microbunch ID, to understand if the increasing hit probability has any influence on the spectrum.

In [ ]:
axes = ["dldTimeSteps", "pulseId"]
bins = [200, 800]
ranges = [(8000, 28000), (0, 800)]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)
plt.figure()
res.plot(robust=True)

We see that the background below the Au 4f core levels slightly changes with microbunch ID. The origin of this is not quite clear yet.

In [ ]:
plt.figure()
(res.loc[{"pulseId":slice(0,50)}].sum(axis=1)/res.loc[{"pulseId":slice(0,50)}].sum(axis=1).mean()).plot()
(res.loc[{"pulseId":slice(700,750)}].sum(axis=1)/res.loc[{"pulseId":slice(700,750)}].sum(axis=1).mean()).plot()
plt.legend(("mbID=0-50", "mbID=700-750"))

## Energy Calibration
We now load a bias series, where the sample bias was varied, effectively shifting the energy spectra. This allows us to calibrate the conversion between the digital values of the dld and the energy.

### time-of-flight spectrum
to compare with what we see on the measurement computer, we might want to plot the time-of-flight spectrum. This is done here.

In [ ]:
sp.append_tof_ns_axis()

Now, to determine proper binning ranges, let's have again a look at the event histograms:

In [ ]:
axes = ["dldTime"]
bins = [150]
ranges = [(-0.5, 150.5)]
sp.view_event_histogram(dfpid=1, axes=axes, bins=bins, ranges=ranges)

### Load energy calibration files
We now load a range of runs sequentially, that were recorded with different sample bias values, and load them afterwards into an xarray

In [ ]:
runs = ["0074", "0073", "0072", "0071", "0070", "0064", "0065", "0066", "0067", "0068", "0069"]
biases = np.arange(962, 951, -1)
data = []
for run in runs:
    sp.load(runs=[run])
    axes = ["dldTimeSteps"]
    bins = [2000]
    ranges = [(1000, 25000)]
    res = sp.compute(bins=bins, axes=axes, ranges=ranges)
    data.append(res)

biasSeries = xr.concat(data, dim=xr.DataArray(biases, dims="sampleBias", name="sampleBias"))

### Load bias series
Now we load the bias series xarray into the processor for calibration

In [ ]:
sp.load_bias_series(binned_data=biasSeries)

### find calibration parameters
We now will fit the tof-energy relation. This is done by finding the maxima of a peak in the tof spectrum, and then fitting the square root relation to obtain the calibration parameters. 

In [ ]:
ranges=(6380, 6700)
ref_id=6
sp.find_bias_peaks(ranges=ranges, ref_id=ref_id, apply=True)

In [ ]:
sp.calibrate_energy_axis(
    ref_id=5,
    ref_energy=-33,
    method="lmfit",
    energy_scale='kinetic',
    d={'value':1.1,'min': .2, 'max':5.0, 'vary':False},
    t0={'value':-1E-8, 'min': -1E-6, 'max': 1e-4, 'vary':True},
    E0={'value': 0., 'min': -1500, 'max': 1500, 'vary': True},
)

### Save calibration
Now we save the calibration parameters into a local configuration file, that will be loaded in the next step

In [ ]:
sp.save_energy_calibration()

## Bin data with energy axis
Now that we have the calibration parameters, we can generate the energy axis for our dataset. We need to load it again, and apply the calibration

In [ ]:
sp.load(runs=np.arange(58, 62))
sp.add_jitter()
sp.filter_column("pulseId", max_value=756)
sp.append_energy_axis()

Now, we can bin as function fo energy and delay stage position

In [ ]:
axes = ['energy', "delayStage"]
bins = [200, 100]
ranges = [[-37,-31], [-135, -115]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges, normalize_to_acquisition_time="delayStage")

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(4, 3), constrained_layout=True)
res.plot(ax=axs)

## Correct delay stage offset.
We can also offset the zero delay of the delay stage

In [ ]:
sp.add_delay_offset(constant=126.9)

In [ ]:
axes = ['energy', "delayStage"]
bins = [200, 100]
ranges = [[-37,-31], [-8, 8]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges, normalize_to_acquisition_time="delayStage")

In [ ]:
res_sub = res - res.loc[{"delayStage": slice(-8, -1)}].mean(axis=1)
fig, axs = plt.subplots(3, 1, figsize=(4, 8), constrained_layout=True)
res.plot(ax=axs[0])
res_sub.plot(ax=axs[1])
res_sub.loc[{"energy":slice(-32.5,-32)}].sum(axis=0).plot(ax=axs[2])